In [59]:
import pandas as pd
import numpy as np
import math

import time

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from tqdm import tqdm

import os

In [60]:
rw = pd.read_excel('./retractions_data/retraction_watch_database.xlsx')
rw.head()

,Record ID,Title,Subject,Institution,Journal,Publisher,Country,Author,URLS,ArticleType,RetractionDate,RetractionDOI,RetractionPubMedID,OriginalPaperDate,OriginalPaperDOI,OriginalPaperPubMedID,RetractionNature,Reason,Paywalled,Notes
0,47271,Binding of DCC by Netrin-1 to Mediate Axon Gui...,(BLS) Biology - Cellular;(BLS) Biology - Gener...,Departments of Anatomy and of Biochemistry and...,Science,American Association for the Advancement of Sc...,United States,Elke Stein;Yimin Zou;Mu-ming Poo;Marc Tessier-...,https://retractionwatch.com/2023/08/31/stanfor...,Research Article;,2023-08-31 00:00:00,10.1126/science.adk1521,0.0,2001-03-09 00:00:00,10.1126/science.1059391,11239160.0,Retraction,+Investigation by Company/Institution;+Manipul...,No,NaN
1,47270,Hierarchical Organization of Guidance Receptor...,(BLS) Biochemistry;(BLS) Biology - General;(BL...,Department of Anatomy and Department of Bioche...,Science,American Association for the Advancement of Sc...,United States,Elke Stein;Marc Tessier-Lavigne,https://retractionwatch.com/2023/08/31/stanfor...,Research Article;,2023-08-31 00:00:00,10.1126/science.adk1517,0.0,2001-02-08 00:00:00,10.1126/science.1058445,11239147.0,Retraction,+Duplication of Image;+Investigation by Compan...,No,NaN
2,47243,Therapeutic potential of targeting IRES-depend...,(BLS) Biochemistry;(BLS) Biology - Cancer;(BLS...,"Division of Hematology-Oncology, UCLA-Greater ...",Oncogene,Springer - Nature Publishing Group,United States,Y Shi;Y Yang;C Bardeleben;B Holmes;J Gera;Alan...,NaN,Research Article;,2023-08-31 00:00:00,10.1038/s41388-023-02820-5,0.0,2015-05-11 00:00:00,10.1038/onc.2015.156,25961916.0,Retraction,+Concerns/Issues About Data;+Concerns/Issues A...,No,see also: https://pubpeer.com/publications/704...
3,47233,A classifier based on 273 urinary peptides pre...,(BLS) Biochemistry;(HSC) Medicine - Cardiovasc...,"Department of Nephrology, The Third Affiliated...",Journal of Hypertension,Wolters Kluwer - Lippincott Williams & Wilkins,China,Lirong Lin;Chunxuan Wang;Jiangwen Ren;Mei Mei;...,NaN,Research Article;,2023-08-30 00:00:00,10.1097/HJH.0000000000003551,37642599.0,2023-08-01 00:00:00,10.1097/HJH.0000000000003467,37199562.0,Retraction,+Concerns/Issues About Results;+Investigation ...,No,see also https://journals.lww.com/jhypertensio...
4,47227,"Age, Gender Demographics and Comorbidity Preva...",(HSC) Biostatistics/Epidemiology;(HSC) Medicin...,"Department of Orthopaedics, Dhanalakshmi Srini...",Journal of Coastal Life Medicine,Journal of Coastal Life Medicine,India,S Venkatesh Kumar;Mohith Singh;Gowtham Singh;K...,NaN,Research Article;,2023-08-30 00:00:00,unavailable,0.0,2023-01-01 00:00:00,unavailable,0.0,Retraction,+Notice - Lack of;+Withdrawal;,No,"date of retraction unknown, article title repl..."


In [61]:
rw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42700 entries, 0 to 42699
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Record ID              42700 non-null  int64  
 1   Title                  42700 non-null  object 
 2   Subject                42700 non-null  object 
 3   Institution            42699 non-null  object 
 4   Journal                42700 non-null  object 
 5   Publisher              42700 non-null  object 
 6   Country                42700 non-null  object 
 7   Author                 42700 non-null  object 
 8   URLS                   21687 non-null  object 
 9   ArticleType            42700 non-null  object 
 10  RetractionDate         42700 non-null  object 
 11  RetractionDOI          42209 non-null  object 
 12  RetractionPubMedID     37599 non-null  float64
 13  OriginalPaperDate      42700 non-null  object 
 14  OriginalPaperDOI       40173 non-null  object 
 15  Or

#### Filtered to contain only articles

In [62]:
article_type = rw[['Record ID', 'ArticleType']].copy()
article_type.loc[:, 'ArticleType'] = article_type['ArticleType'].str.split(';')
article_type = article_type.explode('ArticleType')
article_type = article_type[article_type['ArticleType'].str.strip() != ""]
article_type = article_type[article_type['ArticleType'].str.strip() != ";"]
article_type = article_type['ArticleType'].str.strip()
article_type

0                    Research Article
1                    Research Article
2                    Research Article
3                    Research Article
4                    Research Article
                     ...             
42695                Research Article
42696                  Review Article
42697                  Review Article
42698                  Review Article
42699    Technical Report/White Paper
Name: ArticleType, Length: 44066, dtype: object

In [63]:
article_type.unique()

array(['Research Article', 'Clinical Study', 'Commentary/Editorial',
       'Review Article', 'Meta-Analysis', 'Letter', 'Case Report',
       'Article in Press', 'Preprint', 'Conference Abstract/Paper',
       'Dissertation/Thesis', 'Retraction Notice', 'Retracted Article',
       'Guideline', 'Other', 'Trade Magazines',
       'Technical Report/White Paper', 'Correction/Erratum/Corrigendum',
       'Book Chapter/Reference Work', 'Auto/Biography', 'Revision',
       'Supplementary Materials', 'Government Publication',
       'Expression of Concern', 'Interview/Q&A', 'Legal Case/Analysis'],
      dtype=object)

In [64]:
select_article_type = ['Research Article', 'Clinical Study', 'Commentary/Editorial',
              'Review Article', 'Meta-Analysis', 'Case Report',  'Retracted Article'] 
filtered_df = rw[rw['ArticleType'].str.contains('|'.join(select_article_type))]

In [65]:
filtered_df

,Record ID,Title,Subject,Institution,Journal,Publisher,Country,Author,URLS,ArticleType,RetractionDate,RetractionDOI,RetractionPubMedID,OriginalPaperDate,OriginalPaperDOI,OriginalPaperPubMedID,RetractionNature,Reason,Paywalled,Notes
0,47271,Binding of DCC by Netrin-1 to Mediate Axon Gui...,(BLS) Biology - Cellular;(BLS) Biology - Gener...,Departments of Anatomy and of Biochemistry and...,Science,American Association for the Advancement of Sc...,United States,Elke Stein;Yimin Zou;Mu-ming Poo;Marc Tessier-...,https://retractionwatch.com/2023/08/31/stanfor...,Research Article;,2023-08-31 00:00:00,10.1126/science.adk1521,0.0,2001-03-09 00:00:00,10.1126/science.1059391,11239160.0,Retraction,+Investigation by Company/Institution;+Manipul...,No,NaN
1,47270,Hierarchical Organization of Guidance Receptor...,(BLS) Biochemistry;(BLS) Biology - General;(BL...,Department of Anatomy and Department of Bioche...,Science,American Association for the Advancement of Sc...,United States,Elke Stein;Marc Tessier-Lavigne,https://retractionwatch.com/2023/08/31/stanfor...,Research Article;,2023-08-31 00:00:00,10.1126/science.adk1517,0.0,2001-02-08 00:00:00,10.1126/science.1058445,11239147.0,Retraction,+Duplication of Image;+Investigation by Compan...,No,NaN
2,47243,Therapeutic potential of targeting IRES-depend...,(BLS) Biochemistry;(BLS) Biology - Cancer;(BLS...,"Division of Hematology-Oncology, UCLA-Greater ...",Oncogene,Springer - Nature Publishing Group,United States,Y Shi;Y Yang;C Bardeleben;B Holmes;J Gera;Alan...,NaN,Research Article;,2023-08-31 00:00:00,10.1038/s41388-023-02820-5,0.0,2015-05-11 00:00:00,10.1038/onc.2015.156,25961916.0,Retraction,+Concerns/Issues About Data;+Concerns/Issues A...,No,see also: https://pubpeer.com/publications/704...
3,47233,A classifier based on 273 urinary peptides pre...,(BLS) Biochemistry;(HSC) Medicine - Cardiovasc...,"Department of Nephrology, The Third Affiliated...",Journal of Hypertension,Wolters Kluwer - Lippincott Williams & Wilkins,China,Lirong Lin;Chunxuan Wang;Jiangwen Ren;Mei Mei;...,NaN,Research Article;,2023-08-30 00:00:00,10.1097/HJH.0000000000003551,37642599.0,2023-08-01 00:00:00,10.1097/HJH.0000000000003467,37199562.0,Retraction,+Concerns/Issues About Results;+Investigation ...,No,see also https://journals.lww.com/jhypertensio...
4,47227,"Age, Gender Demographics and Comorbidity Preva...",(HSC) Biostatistics/Epidemiology;(HSC) Medicin...,"Department of Orthopaedics, Dhanalakshmi Srini...",Journal of Coastal Life Medicine,Journal of Coastal Life Medicine,India,S Venkatesh Kumar;Mohith Singh;Gowtham Singh;K...,NaN,Research Article;,2023-08-30 00:00:00,unavailable,0.0,2023-01-01 00:00:00,unavailable,0.0,Retraction,+Notice - Lack of;+Withdrawal;,No,"date of retraction unknown, article title repl..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42694,825,On the Primary Site of Nuclear RNA Synthesis,(BLS) Biology - Cellular;(BLS) Biology - Gener...,Department of Biology University of Pennsylvan...,The Journal of Cell Biology,Rockefeller University Press,United States,Louise M Luckenbill;Alan S Cohen,http://retractionwatch.com/2013/04/24/authors-...,Research Article;,1966-10-01 00:00:00,10.1083/jcb.31.1.195,5971970.0,1959-10-01 00:00:00,10.1083/jcb.6.2.301,13850774.0,Retraction,+Error in Results and/or Conclusions;+Results ...,No,NaN
42695,10433,Change of Venue and the Conflict of Laws,(SOC) Law/Legal Issues;,University of Chicago;,The University of Chicago Law Review,University of Chicago Law School,United States,Brainerd Currie,NaN,Research Article;,1960-02-01 00:00:00,NaN,NaN,1955-02-01 00:00:00,NaN,NaN,Retraction,+Error in Results and/or Conclusions;+Unreliab...,Yes,NaN
42696,18529,Sinn und Grenzen der exakten Wissenschaft,(PHY) Physics;,Berlin,Die Naturwissenschaften,Springer,Germany,Max Planck,NaN,Review Article;,1942-02-01 00:00:00,10.1007/BF01475382,0.0,1942-02-01 00:00:00,10.1007/BF01475382,0.0,Retraction,+Copyright Claims;,No,NaN
42697,18531,Naturwissens

In [66]:
doi_series = filtered_df['OriginalPaperDOI']
len(doi_series)

29266

In [67]:
doi_series.dropna(inplace=True)
doi_series

C:\Users\isabe\AppData\Local\Temp\ipykernel_12872\1982765895.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  doi_series.dropna(inplace=True)


0              10.1126/science.1059391
1              10.1126/science.1058445
2                 10.1038/onc.2015.156
3         10.1097/HJH.0000000000003467
4                          unavailable
                     ...              
42693    10.1016/S0020-1693(00)93130-X
42694              10.1083/jcb.6.2.301
42696               10.1007/BF01475382
42697               10.1007/BF01488952
42698                      unavailable
Name: OriginalPaperDOI, Length: 28943, dtype: object

In [68]:
import glob
import pyarrow.parquet as pq

In [69]:
rwd_wos = pq.read_table("../thesis_data/processed_data/WoS_RWD_DOIS.parquet").to_pandas()

In [70]:
rwd_wos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21054 entries, 0 to 21053
Data columns (total 73 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   AU        21054 non-null  object 
 1   AF        21051 non-null  object 
 2   CR        20943 non-null  object 
 3   AB        19481 non-null  object 
 4   AR        6038 non-null   object 
 5   BE        161 non-null    object 
 6   BN        110 non-null    object 
 7   BP        14782 non-null  object 
 8   C1        20491 non-null  object 
 9   C3        18706 non-null  object 
 10  CA        54 non-null     object 
 11  CL        403 non-null    object 
 12  CT        403 non-null    object 
 13  CY        403 non-null    object 
 14  DA        21054 non-null  object 
 15  DE        13002 non-null  object 
 16  DI        21054 non-null  object 
 17  DT        21054 non-null  object 
 18  EA        2257 non-null   object 
 19  EF        724 non-null    object 
 20  EI        17538 non-null  ob

In [71]:
rename_columns = {
    'AU': "authors", 
    'DE': "author_keywords", 
    'ID': "keywords_plus", 
    'C1': "author_address", 
    'CR': "cited_references", 
    'AB': "abstract", 
    'PA': "publisher_address", 
    #'affiliations', 
    'AR': "article_number", 
    'EM': "email_address",
    'book.author': "book_author", 
    ###'BO': "book",  -> n está na documentação
    'da': "date_report_generated", 
    'DI': "doi", 
    'GA': "document_delivery_number", 
    #'eissn',
    'esi.highly.cited.paper': "esi_highly_cited_paper", 
    'esi.hot.paper': "esi_hot_paper", 
    'earlyaccessdate': "early_access_date", 
    'BE': "editors",
    'FU': "funding_agency_and_grant_number", 
    'FX': "funding_text", 
    'BN': "isbn", 
    'SN': "issn", 
    'JI': "iso_source_abv", 
    'SO': "publication_name", 
    'LA': "language", 
    #'month', 
    #'note', 
    'NR': "cited_reference_count", 
    'PN': "part_number",
    'oa': "open_access_indicator", 
    'orcid.numbers': "orcid_numbers", 
    #'organization', 
    ###'PP', -> n está na documentação
    'PU': "publisher", 
    'SC': "research_areas",
    'researcherid.numbers': "researcher_id_numbers", 
    'SE': "book_series_title", 
    'TC': "wos_core_collection_times_cited_count", 
    'TI': "document_title", 
    'DT': "document_type", 
    'UT': "accession_number",
    'usage.count.last.180.days': "usage_count_last_180_days", 
    'U2': "usage_count_since_2013", 
    'VL': "volume", 
    'web.of.science.categories.': "wos_categories",
    'web.of.science.index': "wos_index", 
    'PY': "year_published", 
    'RP': "reprint_address", 
    'DB': "database", 
    'J9': "29_character_source_abv", 
    'AU_UN': "authors_affiliations", 
    'AU1_UN': "corresponding_author_affiliation",
    'AU_UN_NR': "not_recognized_affiliations", 
    'SR_FULL': "short_full_reference", 
    'SR': "short_reference"
}

rwd_wos.rename(columns = rename_columns, inplace = True)
rwd_wos.columns

Index(['authors', 'AF', 'cited_references', 'abstract', 'article_number',
       'editors', 'isbn', 'BP', 'author_address', 'C3', 'CA', 'CL', 'CT', 'CY',
       'DA', 'author_keywords', 'doi', 'document_type', 'EA', 'EF', 'EI',
       'email_address', 'EP', 'ER', 'funding_agency_and_grant_number',
       'funding_text', 'document_delivery_number', 'GP', 'HC', 'HO', 'HP',
       'keywords_plus', 'IS', '29_character_source_abv', 'iso_source_abv',
       'language', 'MA', 'cited_reference_count', 'OA', 'OI',
       'publisher_address', 'PD', 'PG', 'PI', 'PM', 'part_number', 'PT',
       'publisher', 'year_published', 'RI', 'reprint_address',
       'research_areas', 'book_series_title', 'SI', 'issn', 'publication_name',
       'SP', 'SU', 'wos_core_collection_times_cited_count', 'document_title',
       'U1', 'usage_count_since_2013', 'accession_number', 'volume', 'WC',
       'WE', 'Z9', 'database', 'authors_affiliations',
       'corresponding_author_affiliation', 'not_recognized_affili

In [72]:
rwd_wos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21054 entries, 0 to 21053
Data columns (total 73 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   authors                                21054 non-null  object 
 1   AF                                     21051 non-null  object 
 2   cited_references                       20943 non-null  object 
 3   abstract                               19481 non-null  object 
 4   article_number                         6038 non-null   object 
 5   editors                                161 non-null    object 
 6   isbn                                   110 non-null    object 
 7   BP                                     14782 non-null  object 
 8   author_address                         20491 non-null  object 
 9   C3                                     18706 non-null  object 
 10  CA                                     54 non-null     object 
 11  CL

In [73]:
rw.columns

Index(['Record ID', 'Title', 'Subject', 'Institution', 'Journal', 'Publisher',
       'Country', 'Author', 'URLS', 'ArticleType', 'RetractionDate',
       'RetractionDOI', 'RetractionPubMedID', 'OriginalPaperDate',
       'OriginalPaperDOI', 'OriginalPaperPubMedID', 'RetractionNature',
       'Reason', 'Paywalled', 'Notes'],
      dtype='object')

In [74]:
rw_merged = rw.merge(rwd_wos, how="inner", left_on="OriginalPaperDOI", right_on= "doi")
rw_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20704 entries, 0 to 20703
Data columns (total 93 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Record ID                              20704 non-null  int64  
 1   Title                                  20704 non-null  object 
 2   Subject                                20704 non-null  object 
 3   Institution                            20703 non-null  object 
 4   Journal                                20704 non-null  object 
 5   Publisher                              20704 non-null  object 
 6   Country                                20704 non-null  object 
 7   Author                                 20704 non-null  object 
 8   URLS                                   8029 non-null   object 
 9   ArticleType                            20704 non-null  object 
 10  RetractionDate                         20704 non-null  object 
 11  Re

In [75]:
rw_merged['issn'].nunique()

4077

In [76]:
rw_merged['issn'].unique()

array(['0036-8075', '0950-9232', '0263-6352', ..., '0021-9002',
       '0095-9901', '0028-1042'], dtype=object)

In [77]:
rw_merged['issn'] = rw_merged['issn'].str.replace('-', '')

In [78]:
journals = pd.read_csv('../scimagojr_2022.csv', sep=';')
journals.head(5)

,Rank,Sourceid,Title,Type,Issn,SJR,SJR Best Quartile,H index,Total Docs. (2022),Total Docs. (3years),...,Total Cites (3years),Citable Docs. (3years),Cites / Doc. (2years),Ref. / Doc.,Country,Region,Publisher,Coverage,Categories,Areas
0,1,28773,Ca-A Cancer Journal for Clinicians,journal,"15424863, 00079235","86,091",Q1,198,44,118,...,30318,85,"299,99","97,00",United States,Northern America,Wiley-Blackwell,1950-2022,Hematology (Q1); Oncology (Q1),Medicine
1,2,29431,Quarterly Journal of Economics,journal,"00335533, 15314650","36,730",Q1,292,36,122,...,2141,122,"14,83","66,61",United Kingdom,Western Europe,Oxford University Press,1886-2022,Economics and Econometrics (Q1),"Economics, Econometrics and Finance"
2,3,20315,Nature Reviews Molecular Cell Biology,journal,"14710072, 14710080","34,201",Q1,485,121,328,...,13331,156,"35,47","89,29",United Kingdom,Western Europe,Nature Publishing Group,2000-2022,Cell Biology (Q1); Molecular Biology (Q1),"Biochemistry, Genetics and Molecular Biology"
3,4,18434,Cell,journal,"00928674, 10974172","26,494",Q1,856,420,1637,...,67791,1440,"43,80","65,74",United States,Northern America,Cell Press,1974-2022,"Biochemistry, Genetics and Molecular Biology (...","Biochemistry, Genetics and Molecular Biology"
4,5,15847,New England Journal of Medicine,journal,"00284793, 15334406","26,015",Q1,1130,1410,4561,...,133956,1854,"33,93","10,21",United States,Northern America,Massachussetts Medical Society,1945-2022,Medicine (miscellaneous) (Q1),Medicine


In [79]:
journals['SJR Best Quartile'].value_counts()

SJR Best Quartile
Q1    7186
Q2    5393
Q3    3735
Q4    1702
-       20
Name: count, dtype: int64

In [83]:
quartiles = journals[['Issn','SJR Best Quartile']]
quartiles.loc[:, 'Issn'] = quartiles['Issn'].str.split(',')
quartiles = quartiles.explode('Issn')
quartiles['Issn'] = quartiles['Issn'].str.replace(' ', '')
quartiles = quartiles[quartiles['Issn'].str.strip() != ""]
quartiles


,Issn,SJR Best Quartile
0,15424863,Q1
0,00079235,Q1
1,00335533,Q1
1,15314650,Q1
2,14710072,Q1
...,...,...
18031,18780814,-
18032,01795953,-
18033,00428779,-
18034,18293824,-


In [82]:
quartiles.to_excel('quartiles.xlsx')

In [84]:
rw_merged = rw_merged.merge(quartiles, how='left', left_on='issn', right_on='Issn')

In [85]:
rw_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20704 entries, 0 to 20703
Data columns (total 95 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Record ID                              20704 non-null  int64  
 1   Title                                  20704 non-null  object 
 2   Subject                                20704 non-null  object 
 3   Institution                            20703 non-null  object 
 4   Journal                                20704 non-null  object 
 5   Publisher                              20704 non-null  object 
 6   Country                                20704 non-null  object 
 7   Author                                 20704 non-null  object 
 8   URLS                                   8029 non-null   object 
 9   ArticleType                            20704 non-null  object 
 10  RetractionDate                         20704 non-null  object 
 11  Re

In [86]:
rw_merged['SJR Best Quartile'].value_counts()

SJR Best Quartile
Q1    9998
Q2    6318
Q3    1219
Q4     240
-      234
Name: count, dtype: int64

In [95]:
rw_merged.groupby('SJR Best Quartile')['issn'].nunique()

SJR Best Quartile
-        2
Q1    2007
Q2    1222
Q3     450
Q4      82
Name: issn, dtype: int64

In [91]:
rw_merged['issn'].nunique() - rw_merged['Issn'].nunique()

314

In [92]:
rw_merged['Issn'].nunique()

3763

### Scrape the DOIs

In [87]:
start_doi = 0
end_doi = len(doi_series)
website_path = "https://www.webofscience.com/wos/woscc/summary/3e49766e-1c18-4edf-8d19-1ab46e85eb71-ccdba0d9/relevance/1"

In [88]:
# path for chrome driver
path = Service(ChromeDriverManager().install())

# create the driver
driver = webdriver.Chrome(service = path)

# open chromedriver window
driver.get(website_path)
time.sleep(10)

driver.maximize_window()
time.sleep(3)

# reject cookies
driver.find_element('xpath', '//*[@id="onetrust-reject-all-handler"]').click()

In [26]:
driver.get(website_path)

In [19]:
from selenium.common.exceptions import ElementClickInterceptedException

In [20]:
doi_series

0              10.1126/science.1059391
1              10.1126/science.1058445
2                 10.1038/onc.2015.156
3         10.1097/HJH.0000000000003467
4                          unavailable
                     ...              
42693    10.1016/S0020-1693(00)93130-X
42694              10.1083/jcb.6.2.301
42696               10.1007/BF01475382
42697               10.1007/BF01488952
42698                      unavailable
Name: OriginalPaperDOI, Length: 28943, dtype: object

In [21]:
exception = False

for first_doi_range in tqdm(range(start_doi, end_doi, 40)):
    try: 
        # click on search bar
        driver.find_element('xpath', '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-base-summary-component/app-search-friendly-display/div[1]/app-general-search-friendly-display/app-query-modifier/div/div[1]/div').click()
    except:
        time.sleep(30)
        driver.find_element('xpath', '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-base-summary-component/app-search-friendly-display/div[1]/app-general-search-friendly-display/app-query-modifier/div/div[1]/div').click()

    if exception: 
        # click on search bar
        driver.find_element('xpath', '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-base-summary-component/app-search-friendly-display/div[1]/app-general-search-friendly-display/app-query-modifier/div/div[1]/div').click()

    try: 
        # clear query box
        driver.find_element('xpath','//*[@id="advancedSearchInputArea"]').clear()
    except:
        time.sleep(3)
        driver.find_element('xpath','//*[@id="advancedSearchInputArea"]').clear()

    # select 10 DOIs at a time and create query
    time.sleep(0.5)
    selected_dois = doi_series[first_doi_range:first_doi_range+40]
    query = 'DO=("' + '" OR "'.join(map(str, selected_dois)) + '")'
    driver.find_element('xpath','//*[@id="advancedSearchInputArea"]').send_keys(query)

    # click search 
    driver.find_element('xpath', '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-base-summary-component/app-search-friendly-display/div[1]/app-general-search-friendly-display/app-query-modifier/div/div[1]/div/div[4]/div[2]/app-advanced-search-form/form/div[2]/div[1]/div/div/button[2]/span[1]').click()
    time.sleep(3)

    # calculate total number of publications
    n_publications = int(driver.find_element(By.CLASS_NAME, "brand-blue").text.replace(',', ''))

    # open "Export" options
    try: 
        driver.find_element('xpath', '//*[@id="snRecListTop"]/app-export-menu/div/button').click()
        exception = False
        time.sleep(2)
    except ElementClickInterceptedException:
        print("The DOIs from range ", first_doi_range, " to ", first_doi_range+40, " are not available")
        exception = True
        time.sleep(5)
        continue

    # export to plaintext file
    driver.find_element('xpath', '//*[@id="exportToFieldTaggedButton"]').click()
    

    #default is export all records on page and content that includes issn. 
    time.sleep(2)

    # dropdown "Record Content:"
    driver.find_element('xpath', '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route[1]/app-export-overlay/div/div[3]/div[2]/app-export-out-details/div/div[2]/form/div/div[1]/wos-select/button').click()

    time.sleep(2)
    
    # choose the record content to be "Full Record and Cited References"
    driver.find_element('xpath', '//*[@id="global-select"]/div/div/div[4]/span').click()

    time.sleep(2)
    
    # click export
    driver.find_element('xpath', '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route[1]/app-export-overlay/div/div[3]/div[2]/app-export-out-details/div/div[2]/form/div/div[2]/button[1]').click()

    time.sleep(13)

  0%|          | 0/724 [00:00<?, ?it/s]

100%|██████████| 724/724 [5:19:56<00:00, 26.51s/it]  


In [22]:
doi_series

0              10.1126/science.1059391
1              10.1126/science.1058445
2                 10.1038/onc.2015.156
3         10.1097/HJH.0000000000003467
4                          unavailable
                     ...              
42693    10.1016/S0020-1693(00)93130-X
42694              10.1083/jcb.6.2.301
42696               10.1007/BF01475382
42697               10.1007/BF01488952
42698                      unavailable
Name: OriginalPaperDOI, Length: 28943, dtype: object

In [23]:
selected_dois

42652                      10.1038/272829a0
42653                    10.1007/BF00410641
42654                      10.1038/277646a0
42656    10.1111/j.1471-4159.1976.tb07010.x
42658                    10.1042/bst0010849
42659                     10.1042/bj1480557
42660                     10.1042/bj1480197
42661                     10.1042/bj1480567
42662          10.1016/0014-5793(75)80344-9
42663                      10.1038/262311a0
42664                      10.1038/256057a0
42665          10.1016/0014-5793(76)80541-8
42683           10.1056/NEJM197306212882502
42684         10.1126/science.179.4078.1129
42685                           unavailable
42686                      10.1038/219153b0
42691          10.1126/science.141.3584.919
42692                     10.1038/2121581b0
42693         10.1016/S0020-1693(00)93130-X
42694                   10.1083/jcb.6.2.301
42696                    10.1007/BF01475382
42697                    10.1007/BF01488952
42698                           

In [24]:
doi_series[130:140].values

array(['10.1155/2022/4297888', '10.1155/2022/6171698',
       '10.1155/2022/4052994', '10.1155/2022/1338392',
       '10.1155/2022/3737998', '10.1155/2021/8601425',
       '10.1155/2022/4301761', '10.1155/2022/6793078',
       '10.1155/2022/6127601', '10.1155/2022/5070659'], dtype=object)